In [ ]:
%pip install --upgrade setuptools
%pip install --upgrade gradio

In [ ]:
import json
import setuptools

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from dotenv import dotenv_values
from typechat import Failure, create_language_model
from examples.math.program import TypeChatProgramTranslator, TypeChatProgramValidator, evaluate_json_program
from examples.math import schema as math

In [ ]:
env_vals = dotenv_values()
model = create_language_model(env_vals)
validator = TypeChatProgramValidator()
translator = TypeChatProgramTranslator(model, validator, math.MathAPI)

In [ ]:
import pandas
async def handleCall(func:str, args: list[int|float]) -> int|float:
    print(f"{func}({json.dumps(args)}) ")
    match func:
        case "add":
            return args[0] + args[1]
        case "sub":
            return args[0] - args[1]
        case "mul":
            return args[0] * args[1]
        case "div":
            return args[0] / args[1]
        case "neg":
            return -1 * args[0]
        case "id":
            return args[0]
        case _:
            raise ValueError(f'Unexpected function name {func}')
        
async def get_translation(message, history):
    result = await translator.translate(message)
    if isinstance(result, Failure):
        return f"Translation Failed ❌ \n Context: {result.message}"
    else:
        result = result.value
        math_result = await evaluate_json_program(result, handleCall)
        df = pandas.DataFrame.from_dict(result["@steps"])
        return f"Translation Succeeded! ✅\n Here is a table of operations needed to get the answer \n ``` {df.fillna('').to_markdown(tablefmt='grid')} \n ```  \n Math Result: {math_result}"


def get_examples():
    example_prompts = []
    with open('../examples/math/input.txt') as prompts_file:
        for line in prompts_file:
            example_prompts.append(line)
    return example_prompts

In [ ]:
import gradio as gr

gr.ChatInterface(get_translation, title="🧮 Math", examples=get_examples()).launch()
